### Validation
Validating a credit card involves the Luhn algorithm works as follows:
1. The Luhn algorithm starts from the last digit which is called the check digit. Then moving left from this check digit (←), double the value of every digit at even indices.
2. If the result of this doubling operation is greater than 9 (e.g., 6 × 2 = 12), then subtract 9 from the result (e.g., 12: 12 − 9 = 3) or, equivalently, add the digits of the result (e.g., 12: 1 + 2 =3).
3. Now sum all the digits (including the check digit).
4. If the total is divisible by 10 then the number is valid; otherwise, it is not valid.

In [3]:
# Validate the card the old fashioned way, function accepts 1 argument: number, returns true/false plus issuer

def validate(number: str):
    card_number_lst = [int(num) for num in number]
    check_digit = card_number_lst.pop(-1)
    card_number_lst.reverse()
    card_number_lst = [num * 2 if i % 2 == 0 else num for i, num in enumerate(card_number_lst)]
    card_number_lst = [num - 9 if num > 9 else num for num in card_number_lst]
    total = sum(card_number_lst) + check_digit
    return total % 10 == 0
validate('345106858850943')

True

### Card Type
Only looking to determine if the number belongs to these 4 issuers: Visa, MasterCard, American Express, Discover
A card from a given issuer always starts with this number:
- Visa: 4
- MasterCard: 51, 52, 53, 54, 55
- American Express: 34, 37
- Discover: 6011

In [4]:
def card_type(number: str):
    if number[0] == '4':
        return 'Visa'
    elif number[:2] in ['51', '52', '53', '54', '55']:
        return 'MasterCard'
    elif number[:2] in ['34', '37']:
        return 'American Express'
    elif number[:4] == '6011':
        return 'Discover'
    else:
        return 'Unknown'

card_type('345106858850943')

'American Express'

### Generation
To make this efficient and mitigate the need to post my own CC numbers on the internet, I have written the following function that generates a valid credit card number for a given issuer.

In [5]:
import random
visa = ['4']
mastercard = ['51', '52', '53', '54', '55']
discover = ['6011']
amex = ['34', '37']
def generate(issuer: str) -> str:
    if issuer == 'visa':
        return generate_credit_card(visa[0], 16)
    elif issuer == 'mastercard':
        return generate_credit_card(random.choice(mastercard),16)
    elif issuer == 'discover':
        return generate_credit_card(discover[0],16)
    elif issuer == 'amex':
        return generate_credit_card(random.choice(amex),15)
    else:
        return 'Unknown'


def generate_credit_card(prefix: str = '', length: int = 16) -> str:
    if length < 2:
        raise ValueError("Length must be at least 2")
    if len(prefix) > 4:
        raise ValueError("Prefix length must be at most 4")

    # Generate the random part of the credit card number
    random_part_length = length - len(prefix)
    random_part = ''.join(str(random.randint(0, 9)) for _ in range(random_part_length - 1))

    # Combine prefix and random part
    card_number = prefix + random_part
    digits = [int(digit) for digit in card_number]
    odd_digits = digits[-1::-2]
    even_digits = digits[-2::-2]
    checksum = sum(even_digits) + sum(sum(divmod(2 * digit, 10)) for digit in odd_digits)
    check_digit = 9 * checksum % 10

    # Append the check digit to the credit card number
    card_number += str(check_digit)

    return card_number    
    
[generate('visa'),
generate('mastercard'),
generate('discover'),
generate('amex')]

['4581396493527320', '5558663778160390', '6011628875787121', '342862376778874']

In [6]:
# Put it all together
numbers = [generate('visa'),
 generate('mastercard'),
 generate('discover'),
 generate('amex')]
for number in numbers:
    print(f'{number} is a {card_type(number)} card and is {validate(number)}')

4653022489304820 is a Visa card and is True
5402638068667450 is a MasterCard card and is True
6011445780605359 is a Discover card and is True
378316218364663 is a American Express card and is True


### Editorial
This is very easy code to write (I swiped all of it from the web), and it executes very quickly. It is obviously more efficient and simpler than using an LLM for the purpose of validating credit card numbers, however, it requires that someone know python (or coding in general). The directions for verifying a credit card number have been expressed in plain english above, and a person could be given those instructions and verify credit card numbers, no python necessary. A person with no technical expertise could also give these guidelines to an LLM and ask the LLM to verify credit card numbers. There are products coming onto the market that abstract away all the implementation and deployment steps of setting up an LLM to do something, and allow you to create processes out of LLM blocks, passing the output of one in as the input to the next. Rather than each block executing computer code, each block queries an LLM with instructions that can be given in human language. This empowers the non-technical to build functions that can automate tasks. We are going to explore whether an LLM can capably validate CC numbers given instructions in english. We will also see if an LLM can generate a valid CC number given instructions in english.

### Validating with an LLM
#### ChatGPT
As an experiment I tried to get ChatGPT to do this. This was the initial prompt I gave it:
```
you are a function machine. Your purpose is to return true if a credit card number is valid. The following is how you would determine if a credit card number is valid:
Validating a credit card involves the Luhn algorithm works as follows:
1. The Luhn algorithm starts from the last digit which is called the check digit. Then moving left from this check digit (←), double the value of every digit at even indices.
2. If the result of this doubling operation is greater than 9 (e.g., 6 × 2 = 12), then subtract 9 from the result (e.g., 12: 12 − 9 = 3) or, equivalently, add the digits of the result (e.g., 12: 1 + 2 =3).
3. Now sum all the digits (including the check digit).
4. If the total is divisible by 10 then the number is valid; otherwise, it is not valid.


is the following number a valid credit card number?
345106858850943
```
That is a valid credit card number so it should have replied true, instead it replied false and gave this explanation of its thoughts.
```
To determine if the credit card number 345106858850943 is valid using the Luhn algorithm, we need to follow these steps:

Starting from the right (the check digit), double the value of every other digit.
If the doubled value is greater than 9, subtract 9 from it.
Sum all the digits.
If the total sum is divisible by 10, the credit card number is valid.
Let's apply these steps:

Reverse the number: 349058858601543
Double every other digit starting from the second digit from the right (the check digit is not doubled):
3(2) 4 9(2) 0 5(2) 8 8(2) 6 0(2) 1 5(2) 4 3

After doubling and, if necessary, subtracting 9:
3(6) 4 9(4) 0 5(10) 8 8(16) 6 0(2) 1 5(10) 4 3

Sum all the digits:
3 + 6 + 4 + 9 + 4 + 0 + 5 + 1 + 0 + 8 + 8 + 1 + 6 + 6 + 0 + 2 + 1 + 5 + 1 + 0 + 4 + 3 = 82

Check if the sum is divisible by 10:
82 % 10 = 2 (not divisible by 10)

Since the sum is not divisible by 10, the credit card number 345106858850943 is not valid according to the Luhn algorithm.
```
The mistake was that it forgot to remove the checksum digit before reversing the string. After going back and forth I got it to do the process correctly, however, when I gave it another credit card number, it forgot the corrections I had already given it and made the same mistakes again.
### Some open source LLMs

In [7]:
from langchain_community.llms import Ollama
prompt = "Take a deep breath before answering. The following is how you would determine if a credit card number is valid:\nValidating a credit card involves the Luhn algorithm works as follows:\n1. The Luhn algorithm starts from the last digit which is called the check digit. Then moving left from this check digit (←), double the value of every digit at even indices.\n2. If the result of this doubling operation is greater than 9 (e.g., 6 × 2 = 12), then subtract 9 from the result (e.g., 12: 12 − 9 = 3) or, equivalently, add the digits of the result (e.g., 12: 1 + 2 =3).\n3. Now sum all the digits (including the check digit).\n4. If the total is divisible by 10 then the number is valid; otherwise, it is not valid.\n\n\n is the following number valid? " + str(numbers[0])
llm = Ollama(model="llama2")

llm.invoke(prompt)

'To determine if the credit card number 4653022489304820 is valid using the Luhn algorithm, we will follow the steps outlined:\n\n1. The last digit of the credit card number is 9, which we will use as the check digit.\n2. We double the value of every digit at even indices:\n* 4 × 2 = 8\n* 6 × 2 = 12\n* 3 × 2 = 6\n* 0 × 2 = 0\n3. If the result of the doubling operation is greater than 9, we subtract 9 from the result:\n* 12 - 9 = 3\n4. Now we sum all the digits (including the check digit):\n* 4 + 6 + 3 + 0 = 13\n5. Finally, we check if the total is divisible by 10:\n* 13 / 10 = 1.3\nSince the result is not a whole number, the credit card number is not valid.\n\nTherefore, the credit card number 4653022489304820 is invalid.'

In [7]:
from langchain_community.llms import Ollama

llm = Ollama(model="llama2:13b")

llm.invoke(prompt)

"\nOkay, let's take a deep breath and go through this step by step. 😌\n\nSo, the Luhn algorithm starts with the last digit, which is 3. 🔎\n\nStep 1: Double every digit at even indices. 📈\n\n* 4 x 2 = 8 (even index)\n* 6 x 2 = 12 (even index)\n* 3 x 2 = 6 (even index)\n\nStep 2: If the result of doubling is greater than 9, subtract 9. 🔪\n\n* 8 - 9 = -1 (fail)\n* 12 - 9 = 3 (pass)\n* 6 - 9 = -3 (fail)\n\nStep 3: Sum all the digits. 📝\n\n* 4 + 6 + 3 + 8 + 1 + 2 + 5 + 5 + 3 = 37 (pass)\n\nStep 4: If the total is divisible by 10, the number is valid. 🔢\n\nThe total of 37 is indeed divisible by 10, so the credit card number 4060371938874355 is valid! 💰\n\nAnd that's how you use the Luhn algorithm to validate a credit card number! 🎉"

In [8]:
from langchain_community.llms import Ollama

llm = Ollama(model="wizard-math:13b")

llm.invoke(prompt)

'\nStep 1: Identify the check digit\nThe check digit is the last digit of the credit card number, which in this case is 5.\n\nStep 2: Double the value of every digit at even indices\nWe will start from the leftmost digit (3) and double its value: 3 * 2 = 6. Next, we move to the second digit (0), double its value: 0 * 2 = 0. Then, we move to the fourth digit (7), double its value: 7 * 2 = 14, which is greater than 9. So, we subtract 9 from the result: 14 - 9 = 5. We continue this process for all even digits, including the check digit (5).\n\nStep 3: Sum all the digits\nNow, we add up all the digits, both odd and even, along with the check digit: 6 + 0 + 7 + 5 + 5 = 23.\n\nStep 4: Check if the total is divisible by 10\nSince 23 is not divisible by 10 (23 / 10 = 2 with a remainder of 3), the credit card number is not valid according to the Luhn algorithm.\n\nSo, the rewritten answer is: The given credit card number is 4060371938874355. Following the Luhn algorithm, we find that the check 

In [10]:
from langchain_community.llms import Ollama

llm = Ollama(model="wizard-math")
llm.invoke('hey friend')
llm.invoke("Is the following number valid according to the Luhn algorithm? "+ str(numbers[0]))

"The Luhn algorithm is a checksum formula used to validate a variety of identification numbers, such as credit card numbers. The process involves taking the digits of the number in alternating sequence from right to left starting from the second-to-last digit, multiplying each by 2 (or 1 if the result would be 10 or greater), and then adding all the results together. If the final total is a multiple of 10, then the number is valid according to the Luhn algorithm.\n\nTo verify if the given number 4653022489304820 is valid according to the Luhn algorithm, we need to follow these steps:\n\nStep 1: Start from the second-to-last digit and go leftwards in alternating sequence. This means we start with the last digit (0), then the second-last digit (2), then the third-last digit (8), and so on. We will be working with these digits only for now.\n\nStep 2: Multiply each of these digits by 2. If the result is 10 or more, add the two digits together. For example, if we have a digit 'x', we would

In [ ]:
# from langchain_community.llms import Ollama
# 
# llm = Ollama(model="wizard-math:70b-q4_0")
# 
# llm.invoke(prompt)